<a href="https://colab.research.google.com/github/nyoons/ESAA/blob/main/Ch_09_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_%EA%B3%BC%EC%A0%9C3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**06. 아이템 기반 최근접 이웃 협업 필터링 실습**
: 사용자 기반 vs 아이템 기반 중 아이템 기반으로 구현해보자.

###**데이터 가공 및 변환**



In [36]:
import pandas as pd
import numpy as np

movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')
print(movies.shape)
print(ratings.shape)

(9742, 3)
(100836, 4)


ratings는 사용자별로 영화에 대한 평점을 매긴 데이터 세트. 평점은 0.5~5점으로 0.5 단위.

협업 필터링은 이 ratings 데이터 세트와 같이 사용자와 아이템 간의 평점에 기반해 추천하는 시스템. 구현해보자. 먼저, 로우 레벨 형태의 원본 데이터 세트를 모든 사용자를 로우로, 모든 영화를 칼럼으로 구성한 데이터 세트로 변경해야 한다. (사용자-아이템 행렬). pivot_table()을 이용하면 쉽게 변환할 수 있다.

In [37]:
ratings=ratings[['userId', 'movieId', 'rating']]
ratings_matrix=ratings.pivot_table('rating', index='userId', columns='movieId')
ratings_matrix.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


NaN 값이 눈에 띈다. 사용자가 평점을 매기지 않은 영화도 있기 때문. 최소 평점이 0.5 이므로 NaN은 0으로 변환하자. 그 전에, 칼럼명이 현재 숫자로 할당되어 어떤 영화인지 알기 어려우니 칼럼명을 id가 아닌 영화명으로 변경하자. movies 데이터 이용.

In [38]:
#title 칼럼을 얻기 위해 movies와 조인
rating_movies=pd.merge(ratings, movies, on='movieId')

#columns='title'로 title 칼럼으로 피벗 수행.
ratings_matrix=rating_movies.pivot_table('rating', index='userId', columns='title')

#NaN 값 0 변환
ratings_matrix=ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


###**영화 간 유사도 산출**
변환된 사용자-영화 평점 행렬 데이터로 영화 간 유사도 측정하자. 코사인 유사도 기반. 코사인 유사도는 행을 기준으로 서로 다른 행을 비교해 유사도를 산출하므로 ratings_matrix로는 할 수 없다. (그럼 사용자 간의 유사도가 만들어진다). 

영화를 기준으로 적용하려면 현재 ratings_matrix에서 행, 열이 바뀌어야함.

In [39]:
ratings_matrix_T=ratings_matrix.transpose()
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


유사도를 구해보자. 좀 더 직관적으로 보기 위해 코사인 유사도로 반환된 넘파이 행렬에 영화명을 매핑해 df로 변환하자.

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim=cosine_similarity(ratings_matrix_T, ratings_matrix_T)

#코사인 유사도로 반환된 넘파이 행렬을 영화명을 매핑해 df로 변환
item_sim_df=pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)
print(item_sim_df.shape)
item_sim_df.head(3)

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


9719*9719. 영화 '대부'와 유사도가 높은 상위 6개 영화를 추출해보자.

In [41]:
item_sim_df['Godfather, The (1972)'].sort_values(ascending=False)[:6]

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.821773
Goodfellas (1990)                            0.664841
One Flew Over the Cuckoo's Nest (1975)       0.620536
Star Wars: Episode IV - A New Hope (1977)    0.595317
Fargo (1996)                                 0.588614
Name: Godfather, The (1972), dtype: float64

이전에 장르만 비교한 콘텐츠 기반 필터링과 다른 점은 장르가 완전 다른 영화도 유사도가 높게 나타났다는 것.

이번에는 '인셉션'과 유사도가 높은 영화를 찾아보자.

In [42]:
#인셉션은 제외하고 추출
item_sim_df['Inception (2010)'].sort_values(ascending=False)[1:6]

title
Dark Knight, The (2008)          0.727263
Inglourious Basterds (2009)      0.646103
Shutter Island (2010)            0.617736
Dark Knight Rises, The (2012)    0.617504
Fight Club (1999)                0.615417
Name: Inception (2010), dtype: float64

만들어진 아이템 기반 유사도 데이터는 사용자의 평점 정보를 모두 취합해 영화에 따라 유사한 다른 영화를 추천할 수 있게 해준다.

이 아이템 기반 유사도 데이터를 이용해 개인에게 특화된 영화 추천 알고리즘을 만들어보자.

###**아이템 기반 최근접 이웃 협업 필터링으로 개인화된 영화 추천**
영화간의 유사도만 가지고 추천하지 말고, 개인적인 취향도 반영해보자. 개인화된 영화 추천의 가장 큰 특징은 개인이 아직 관람하지 않은 영화를 추천한다는 것. 아직 관람하지 않은 영화 중 아이템 유사도와 기존에 관람한 영화의 평점 데이터를 기반으로 새롭게 모든 영화의 예측 평점을 계산한 후 높은 예측을 가진 영화를 추천.

개인화된 예측 평점을 구하는 식을 참고. 사용자별로 최적화된 평점 스코어를 예측하는 함수를 만들자.

In [43]:
def predict_rating(ratings_arr, item_sim_arr):
  ratings_pred=ratings_arr.dot(item_sim_arr)/np.array([np.abs(item_sim_arr).sum(axis=1)])
  return ratings_pred

개인화된 예측 평점을 구해보자.

In [44]:
ratings_pred=predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix=pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501


예측 평점은 사용자별 영화의 실제 평점과 영화의 코사인 유사도를 내적한 값이기 때문에 기존에 관람하지 않았던 영화 평점 셀에 값이 부여되는 경우가 발생. 예측 평점이 실제보다 작을 수 있다.

이 예측 결과가 실제 평점과 얼마나 차이가 나는지 확인하자. 사용자가 영화의 평점을 주지 않은 경우 0을 넣었던 것을 고려해 실제와 예측 평점 차이는 기존에 평점이 부여된 데이터에 대해서만 MSE를 적용하겠다.

In [45]:
from sklearn.metrics import mean_squared_error

#사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE를 구함
def get_mse(pred, actual):
  #평점이 있는 영화만 추출
  pred=pred[actual.nonzero()].flatten()
  actual=actual[actual.nonzero()].flatten()
  return mean_squared_error(pred, actual)

print('아이템 기반 모든 최근접 이웃 MSE :', get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 최근접 이웃 MSE : 9.895354759094706


실제와 예측은 서로 스케일이 달라 MSE가 클 수 있다. 감소시키는 방향으로 개선하면 됨.

앞의 predict_rating 함수는 사용자별 영화의 예측 평점을 계산하기 위해 해당 영화와 다른 모든 영화 간의 유사도 벡터를 적용한 것. 많다보니 상대적으로 평점 예측이 감소. 특정 영화와 가장 비슷한 유사도를 갖는 영화에 대해서만 유사도 벡터를 적용하도록 함수 변경하자.

In [46]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
  #사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
  pred=np.zeros(ratings_arr.shape)

  #사용자-아이템 평점 행렬의 열 크기만큼 루프 수행
  for col in range(ratings_arr.shape[1]):
    #유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 인덱스 반환
    top_n_items=[np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
    #개인화된 예측 평점을 계산
    for row in range(ratings_arr.shape[0]):
      pred[row, col]=item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T)
      pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))
  
  return pred

새로 만든 함수를 이용해 예측 평점을 계산하고, MSE를 구해보자.

In [47]:
ratings_pred=predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=20)
print('아이템 기반 최근접 top-20 이웃 mse:', get_mse(ratings_pred, ratings_matrix.values))

#계산된 예측 평점 데이터는 df로 재생성
ratings_pred_matrix=pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)

<ipython-input-46-2279deb9f1dc>:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col]=item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T)
<ipython-input-46-2279deb9f1dc>:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))


아이템 기반 최근접 top-20 이웃 mse: 3.6949827608772314


3.69로 기존보다 많이 향상.

이제 특정 사용자에 대해 영화를 추천해보자.

In [48]:
#userId=9인 사용자에 대해 추천해보자

#어떤 영화를 좋아하는가
user_rating_id=ratings_matrix.loc[9, :]
user_rating_id[user_rating_id>0].sort_values(ascending=False)[:10]

title
Adaptation (2002)                                                                 5.0
Citizen Kane (1941)                                                               5.0
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    5.0
Producers, The (1968)                                                             5.0
Lord of the Rings: The Two Towers, The (2002)                                     5.0
Lord of the Rings: The Fellowship of the Ring, The (2001)                         5.0
Back to the Future (1985)                                                         5.0
Austin Powers in Goldmember (2002)                                                5.0
Minority Report (2002)                                                            4.0
Witness (1985)                                                                    4.0
Name: 9, dtype: float64

대작, 어드벤처 등 전반적으로 흥행성이 좋은 영화.

사용자가 이미 평점을 준 영화를 제외하고 추천할 수 있도록 평점을 주지 않은 영화를 리스트 객체로 반환하는 함수 생성하자.

In [49]:
def get_unseen_movies(ratings_matrix, userId):
  #userid로 입력받은 사용자의 모든 영화 정보를 추출해 series로 변환
  #반환된 user_rating은 영화명을 인덱스로 가지는 series 객체
  user_rating=ratings_matrix.loc[userId, :]

  #user_rating이 0보다 크면 기존에 관람한 영화. 대상 인덱스를 추출해 list 객체로 만들자.
  already_seen=user_rating[user_rating>0].index.tolist()

  #모든 영화명을 list 객체로 만듦
  movies_list=ratings_matrix.columns.tolist()

  #list coprehension으로 already_seen에 해당하는 영화는 movies_list에서 제외
  unseen_list=[movie for movie in movies_list if movie not in already_seen]

  return unseen_list

사용자가 평점을 주지 않은 추천 대상 영화 정보와 predict_rating_topsim에서 추출한 사용자별 아이템 유사도에 기반한 예측 평점 데이터 세트를 이용해 최종적으로 사용자에게 영화를 추천하는 함수를 만들자.

In [50]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
  #예측 평점 df에서 사용자 id 인덱스와 unseen_list로 들어온 영화명 칼럼을 추출해 가장 예측 평점이 높은 순으로 정렬
  recomm_movies=pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
  return recomm_movies

#사용자가 관람하지 않은 영화명 추출
unseen_list=get_unseen_movies(ratings_matrix, 9)

#아이템 기반의 최근접 이웃 협업 필터링으로 영화 추천
recomm_movies=recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

#평점 데이터를 df로 생성
recomm_movies=pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])

recomm_movies

,pred_score
title,
Shrek (2001),0.866202
Spider-Man (2002),0.857854
"Last Samurai, The (2003)",0.817473
Indiana Jones and the Temple of Doom (1984),0.816626
"Matrix Reloaded, The (2003)",0.800990
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.765159
Gladiator (2000),0.740956
"Matrix, The (1999)",0.732693
Pirates of the Caribbean: The Curse of the Black Pearl (2003),0.689591


다양하지만 높은 흥행성을 가진 작품들이 추천됨.

##**07. 행렬 분해를 이용한 잠재 요인 협업 필터링 실습**
행렬 분해를 이용한 잠재 요인 협업 필터링을 직접 구현해보자. 일반적으로 행렬 분해에는 SVD가 자주 사용되지만, 사용자-아이템 평점 행렬에는 null값이 많은 문제로 SGD나 ALS 기반의 행렬 분해를 이용. SGD 기반의 행렬 분해를 구현해보자.

행렬 분해 로직 부분을 함수로 만들자.

In [51]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
  error=0
  #두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
  full_pred_matrix=np.dot(P, Q.T)

  #실제 R 행렬에서 널이 아닌 값의 위치 인덱스를 추출해 실제 R 행렬롸 예측 행렬의 RMSE 추출
  x_non_zero_ind=[non_zero[0] for non_zero in non_zeros]
  y_non_zero_ind=[non_zero[1] for non_zero in non_zeros]
  R_non_zeros=R[x_non_zero_ind, y_non_zero_ind]
  full_pred_matrix_non_zeros=full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
  mse=mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
  rmse=np.sqrt(mse)

  return rmse

In [55]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
  num_users, num_items=R.shape
  #P와 Q 매트릭스의 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력
  np.random.seed(1)
  P=np.random.normal(scale=1./K, size=(num_users, K))
  Q=np.random.normal(scale=1./K, size=(num_items, K))

  prev=rmse=10000
  break_count=0

  #R>0인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장.
  non_zeros=[(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j]>0]

  #SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
  for step in range(steps):
    for i, j, r in non_zeros:
      #실제 값과 예측 값의 차이인 오류 값 구함
      eij=r - np.dot(P[i, :], Q[j, :].T)
      #Regularization 을 반영한 SGD 업데이트 공식 적용
      P[i, :]=P[i, :]+learning_rate*(eij*Q[j, :]-r_lambda*P[i, :])
      Q[j, :]=Q[j, :]+learning_rate*(eij*P[i, :]-r_lambda*Q[j, :])

    rmse=get_rmse(R, P, Q, non_zeros)
    if (step%10)==0:
      print('### iteration setp :', step, 'rmse :', rmse)
    
  return P, Q

평점 행렬 데이트 새로 로딩

In [56]:
import pandas as pd
import numpy as np

movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')
ratings=ratings[['userId', 'movieId', 'rating']]
ratings_matrix=ratings.pivot_table('rating', index='userId', columns='movieId')

#title 칼럼을 얻기 위해 movies와 조인 수행
rating_movies=pd.merge(ratings, movies, on='movieId')
#columns='title'로 title 칼럼으로 pivot 수행
ratings_matrix=rating_movies.pivot_table('rating', index='userId', columns='title')

다시 만들어진 사용자-아이템 행렬을 행렬 분해하자.

In [57]:
P, Q=matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda=0.01)
pred_matrix=np.dot(P, Q.T)

### iteration setp : 0 rmse : 2.9023619751336867
### iteration setp : 10 rmse : 0.7335768591017927
### iteration setp : 20 rmse : 0.5115539026853442
### iteration setp : 30 rmse : 0.37261628282537446
### iteration setp : 40 rmse : 0.2960818299181014
### iteration setp : 50 rmse : 0.2520353192341642
### iteration setp : 60 rmse : 0.22487503275269854
### iteration setp : 70 rmse : 0.2068545530233154
### iteration setp : 80 rmse : 0.19413418783028685
### iteration setp : 90 rmse : 0.18470082002720406
### iteration setp : 100 rmse : 0.17742927527209104
### iteration setp : 110 rmse : 0.1716522696470749
### iteration setp : 120 rmse : 0.16695181946871726
### iteration setp : 130 rmse : 0.16305292191997542
### iteration setp : 140 rmse : 0.15976691929679646
### iteration setp : 150 rmse : 0.1569598699945732
### iteration setp : 160 rmse : 0.15453398186715425
### iteration setp : 170 rmse : 0.15241618551077643
### iteration setp : 180 rmse : 0.1505508073962831
### iteration setp : 190 rmse : 

더 쉽게 칼럼을 이해하기 위해 칼럼명을 영화 타이틀로 바꾸자.

In [58]:
ratings_pred_matrix=pd.DataFrame(data=pred_matrix, index=ratings_matrix.index, columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941


예측 사용자-아이템 평점 행렬 정보가 만들어졌다.

이제 개인화된 영화 추천을 해보자. 사용자 아이디 9번에 대한 추천을 이번에는 잠재 요인 협업 필터링으로 추천해보자.

In [59]:
#사용자가 관람하지 않은 영화명 추출
unseen_list=get_unseen_movies(ratings_matrix, 9)

#잠재 요인 협업 필터링으로 영화 추천
recomm_movies=recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

#평점 데이터를 df로 생성
recomm_movies=pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


앞 절의 아이템 기반 협업 필터링 결과와는 많이 다르다.

##**08. 파이썬 추천 시스템 패키지 - Surprise**
###**Surprise 패키지 소개**
지금까지 콘텐츠 기반 필터링, 아이템 기반 협업 필터링, 잠재 요인 기반 협업 필터링을 구현.

파이썬 기반의 추천 시스템 구축을 위한 전용 패키지 : surprise. 장점은 다음과 같다.
- 다양한 추천 알고리즘 (사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링)을 쉽게 적용해 추천 시스템 구축
- fit, predict(), train_test_split, cross_validate, gridsearchcv 등의 클래스를 통해 추천 시스템을 위한 모델 셀렉션 평가, 하이퍼 파라미터 튜닝 등의 기능 제공.

In [61]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095490 sha256=a7d4cb878815a00245f077ad87635e463d9deda72811962ddb45cfd498e25170
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


###**Surprise를 이용한 추천 시스템 구축**
간단한 예제를 통해 surprise 패키지의 개략적인 사용법을 익혀보자.

추천 데이터를 학습용과 테스트용 데이터 세트로 분리한 뒤, SVD 행렬 분해를 통해 잠재 요인 협업 필터링을 수행.

In [62]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

추천을 위한 데이터 세트 로딩. surprise에서 데이터 로딩은 dataset 클래스를 이용해서만 가능하다. surprise는 movie lens 데이터 세트의 사용자-영화 평점 데이터 포맷과 같이 userId, movieId, rating과 같은 주요 데이터가 로우 레벨(-> 방향으로 데이터가 되어 있음) 형태로 되어 있는 포맷의 데이터만 처리한다.

In [63]:
#과거 버전의 데이터임. 최근 영화에 대한 평점 없다.

data=Dataset.load_builtin('ml-100k')
#수행시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset=train_test_split(data,test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


시행하면 내려받기가 된다.

주의해야할 점은 사용자-아이템 평점 데이터를 적용할 때 무비렌즈 사이트에서 내려받은 데이터 파일과 동일하게 로우 레벨의 사용자-아이템 평점 데이터를 그대로 적용해야 한다는 것. 앞에서 로우 레벨의 사용자-아이템 평점 데이터를 아이디를 칼럼명으로 변환한 형태의 사용자-아이템 평점 행렬 데이터로 변환했다. surprise는 자체적으로 로우 레벨의 데이터를 칼럼 레벨의 데이터로 변경하므로 원본 데이터를 데이터 세트로 적용.

이제 SVD로 잠재 요인 협업 필터링을 수행하자.

In [64]:
algo=SVD() #알고리즘 객체 생성
algo.fit(trainset) #이 알고리즘에 fit을 수행해 학습 데이터 세트 기반으로 추천 알고리즘을 학습

학습된 추천 알고리즘을 기반으로 테스트 데이터 세트에 대해 추천을 수행하자. test()(사용자-아이템 평점 데이터 세트 전체에 대해 추천을 예측)와 predict()(개별 사용자와 영화에 대한 추천 평점 반환)이용. 

In [65]:
#test()의 작동
predictions=algo.test(testset)
print('predictions type :', type(predictions), 'size :', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

predictions type : <class 'list'> size : 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.691991813467798, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.705707305506705, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.054191985450693, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.4039506443621894, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.2251276456948292, details={'was_impossible': False})]

In [66]:
#userid, itemid, est에 접근해보자
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.691991813467798),
 ('882', '291', 3.705707305506705),
 ('535', '507', 4.054191985450693)]

In [67]:
#predict()의 작동

#사용자 아이디, 아이템 아이디는 문자열로 입력해야함.
uid=str(196)
iid=str(302)
pred=algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.14   {'was_impossible': False}


predict는 이렇게 개별 사용자와 아이템 정보를 입력하면 추천 예측 평점을 반환.

테스트 데이터 세트를 이용해 추천 예측 평점과 실제 평점과의 차이도 평가해보자.

In [68]:
#바로 rmse 사용
accuracy.rmse(predictions)

RMSE: 0.9492


0.9491839577460947

###**Surprise 주요 모듈 소개**
**Dataset**

로우 레벨이며 네번째 칼럼부터는 아예 로딩을 수행하지 않는 surprise. 따라서 데이터 세트의 칼럼 순서가 반드시 사용자 아이디, 아이템 아이디, 평점 순으로 되어 있어야 한다.

**OS 파일 데이터를 Surprise 데이터 세트로 로딩**

로딩 되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있어서는 안된다. ratings 파일에 있으므로 to_csv() 함수로 헤더를 삭제하자.

In [69]:
import pandas as pd

ratings=pd.read_csv('ratings.csv')
#ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성.
ratings.to_csv('ratings_noh.csv', index=False, header=False)

데이터 파일의 파싱 포맷 정의.

reader 객체 생성 시 line_foramt 인자로 user, item, rating, timestamp의 4개 칼럼으로 데이터가 구성되어 있음을 명시. 각 칼럼의 분리 문자는 콤마, 평점 단위는 0.5, 최대 평점은 5로 설정. reader 설정을 완료하면 이를 기반으로 데이터를 파싱하며 로딩한다.

In [71]:
from surprise import Reader

reader=Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('ratings_noh.csv', reader=reader)

- line_format(string) : 칼럼을 순서대로 나열. 입력된 문자열을 공백으로 분리해 칼럼으로 인식.

이제 SVD 행렬 분해 기법을 이용해 추천을 예측해보자.

In [72]:
trainset, testset=train_test_split(data, test_size=0.25, random_state=0)

#수행시마다 동일한 결과를 도출하기 위해 random_state 설정
algo=SVD(n_factors=50, random_state=0)

#학습 데이터 세트로 학습하고 테스트 데이터 세트로 평점 예측 후 rmse 평가
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

**판다스 DF에서 Surprise 데이터 세트로 로딩**

판다스의 df에서도 surprise 데이터 세트로 로딩할 수 있지만 주의할 점은 역시 사용자 아이디, 아이템 아이디, 평점 칼럼 순서.

In [73]:
import pandas as pd
from surprise import Reader, Dataset

ratings=pd.read_csv('ratings.csv')
reader=Reader(rating_scale=(0.5, 5.0))

#ratings df에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서
data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset=train_test_split(data, test_size=0.25, random_state=0)

algo=SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

###**Surprise 추천 알고리즘 클래스**
surprise SVD의 비용 함수는 사용자 베이스라인 편향성을 감안한 평점 예측에 regularization을 적용한 것. SVD 클래스의 입력 파라미터 n_factors와 n_epochs의 값을 변경해 튜닝할 수 있으나 튜닝 효과는 크지 않다.

추천 알고리즘 예측 성능 벤치마크 결과를 살펴보자. SVD++ 알고리즘의 성적이 가장 좋지만, 시간이 오래 걸린다. SVD++을 제외하면 SVD, k-NN baseline이 가장 성적이 좋다.

###**베이스라인 평점**
k-nn baseline에서 baseline이란??

평가가 각 개인의 성향에 따라(냉정하게 해야지, 후하게 해야지) 달라질 수 있다는 경향을 반영해 아이템 평가에 편향성 요소를 반영하게 평점을 부과하는 것을 베이스라인 평점이라고 함.

전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 공식으로 계산된다. 이 때
- 사용자 편향 점수=사용자별 아이템 평점 평균 값 - 전체 평균 평점
- 아이템 편향 점수=아이템별 평점 평균 값 - 전체 평균 평점

이다.


###**교차 검증과 하이퍼 파라미터 튜닝**
surprise는 cross_validate()와 gridsearchcv를 제공.

In [74]:
#cross_validate() 작동
from surprise.model_selection import cross_validate

#판다스 df에서 surprise 데이터 세트로 데이터 로딩
ratings=pd.read_csv('ratings.csv')
reader=Reader(rating_scale=(0.5, 5.0))
data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo=SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8724  0.8729  0.8735  0.8726  0.8834  0.8750  0.0042  
MAE (testset)     0.6709  0.6706  0.6680  0.6725  0.6800  0.6724  0.0041  
Fit time          1.74    2.57    1.94    2.98    2.34    2.32    0.44    
Test time         0.42    0.27    0.21    0.15    0.28    0.27    0.09    


{'test_rmse': array([0.8724035 , 0.87288946, 0.87345643, 0.87262857, 0.88337819]),
 'test_mae': array([0.67085041, 0.67056714, 0.66798929, 0.67254846, 0.6800387 ]),
 'fit_time': (1.742537260055542,
  2.572179079055786,
  1.943324089050293,
  2.981207847595215,
  2.3396496772766113),
 'test_time': (0.41981959342956543,
  0.2744271755218506,
  0.20737552642822266,
  0.15221929550170898,
  0.28346729278564453)}

폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치를 함께 보여준다.

In [75]:
#gridsearchcv 작동
from surprise.model_selection import GridSearchCV

#최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid={'n_epochs':[20, 40, 60],
            'n_factors':[50, 100, 200]}

#CV를 3개 폴드 세트로 지정, 성능 평가를 rmse, mse로 수행하도록 gridsearchcv 구성
gs=GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

#최고 RMSE evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8783763629335608
{'n_epochs': 20, 'n_factors': 50}


###**Surprise를 이용한 개인화 영화 추천 시스템 구축**
surprise 패키지로 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은 영화 중에서 개인 취향에 가장 적절한 영화를 추천해보자. 따라서 ratings 데이터를 분리하지 않고 전체를 학습 데이터로 사용한다.

다만, surprise는 데이터 세트를 split()을 이용해 내부에서 사용하는 trainset 클래스 객체로 변환하지 않으면 fit으로 학습할 수가 없다.

In [ ]:
#따라서 분리하지 않은 데이터 세트에 fit을 수행한 이 코드는 오류 발생

#data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
#algo=SVD(n_factors=50, random_state=0)
#algo.fit(data)

데이터 전체를 학습 데이터로 사용하려면 DatasetAutoFolds 클래스를 이용한다.

In [76]:
from surprise.dataset import DatasetAutoFolds

reader=Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
#datasetaufofolds 클래스를 ratings_noh.csv 기반으로 생성
data_folds=DatasetAutoFolds(ratings_file='ratings_noh.csv', reader=reader)

#전체 데이터를 학습 데이터로 생성
trainset=data_folds.build_full_trainset()

In [77]:
#build_full_trainset을 이용해 생성된 학습 데이터를 기반으로 학습 수행
algo=SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

특정 사용자는 9번 사용자로 지정. 간단하게 예제를 수행하기 위해 9번 사용자가 아직 평점을 매기지 않은 영화를 movieId 42로 선정한 뒤 예측 평점 계산해보자.

In [78]:
#영화에 대한 상세 속성 정보 df 로딩
movies=pd.read_csv('movies.csv')

#userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인
movieIds=ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count()==0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [79]:
#predict() 이용하면 되겠지
uid=str(9)
iid=str(42)

pred=algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


이제 사용자가 평점을 매기지 않은 전체 영화를 추출한 뒤에 예측 평점 순으로 영화를 추천해보자.

먼저 대상이 되는 영화 추출.

In [81]:
def get_unseen_surprise(ratings, movies, userId):
  #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies=ratings[ratings['userId']==userId]['movieId'].tolist()

  #모든 영화의 movieId를 리스트로 생성
  total_movies=movies['movieId'].tolist()

  #모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies=[movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies), '전체 영화 수:', len(total_movies))

  return unseen_movies

unseen_movies=get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


앞에서 학습된 추천 알고리즘 클래스인 SVD를 이용해 높은 예측 평점을 가진 순으로 영화를 추천해보자.

In [82]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
  #알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list로 저장
  predictions=[algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  #predictions list 객체는 surprise의 predictions 객체를 원소로 갖고 있다.
  # [Prediction(uid='9', iid='1', est=3.69), Predictions(uid='9', iid='2', est=2.98),,,]

  #이를 est 값으로 정렬하기 위해 아리애 sortkey_est 함수 정의
  #sortdey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
  def sortkey_est(pred):
    return pred.est

  #sortkey_est() 반환값의 내림차순으로 정렬 수행하고 top_n개의 최상위 값 추출
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions=predictions[:top_n]

  #top_n 으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids=[int(pred.iid) for pred in top_predictions]
  top_movie_rating=[pred.est for pred in top_predictions]
  top_movie_titles=movies[movies.movieId.isin(top_movie_ids)]['title']
  top_movie_preds=[ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

  return top_movie_preds

unseen_movies=get_unseen_surprise(ratings, movies, 9)
top_movie_preds=recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('### top-10 추천 영화 리스트 ###')
for top_movie in top_movie_preds:
  print(top_movie[1], ':', top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
### top-10 추천 영화 리스트 ###
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122657
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


surprise 사용 시 복잡한 알고리즘을 구현할 필요 없이 쉽고 간결하게 추천 시스템을 구현할 수 있다. 